In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time

url='https://hiking.biji.co/'
driver_path=Service('D:/chromedriver.exe')
driver=webdriver.Chrome(service=driver_path)
driver.get(url)
time.sleep(2)

hh=driver.find_elements(By.CSS_SELECTOR,'.justify-center')[0].find_element(By.CSS_SELECTOR,'a[title="路線"]')
hh.click()
oo=driver.find_element(By.CSS_SELECTOR,'.justify-center li>a[href="/index.php?q=mountain"]')
oo.click()
time.sleep(5)
driver.close()

In [ ]:
# 全部爬完變df再存到csv

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import bs4,os,time
import pandas as pd

# 啟動瀏覽器工具的選項
options = webdriver.ChromeOptions()
# options.add_argument("--headless")                # 不開啟實體瀏覽器背景執行
options.add_argument("--incognito")               # 開啟無痕模式
options.add_argument("--disable-popup-blocking") # 禁用彈出攔截

#page_imgs_list,page_contents_list=[],[]
start_page=127 # 如果中途斷掉 將此改為斷掉的頁數 並註解上方page_contents_list

url='https://hiking.biji.co/index.php?q=mountain'
#'&page='
driver_path=Service('D:/chromedriver.exe')
driver=webdriver.Chrome(service=driver_path)
driver.get(url)

# 去抓個頁面的內容
def Get_page_content_fn():
    driver_soup=bs4.BeautifulSoup(driver.page_source,'lxml')  # 在def外就只會一直抓第一頁
    find_imgs=driver_soup.find('div',id='mountain_list').find_all('div','flex')
    find_flexs=driver_soup.find('div',id='mountain_list').find_all('div','flex-1')  # 要注意div內是否還有一樣的屬性名稱
    print(len(find_flexs))
    print(len(find_imgs))
    
    for i in range(0,len(find_imgs),2): 
        img_url=find_imgs[i].a.img['data-src']  # 選擇完整網址的屬性'data-src' 比'src'不易有bug
        #print(i,img_url)
        page_imgs_list.append(img_url)
    for flex,img_ in zip(find_flexs,page_imgs_list):  # 放入檔案後要清空放照片的page_imgs_list 才不會永遠只抓前20筆
        #print(flex.text.strip())
        title=flex.h2.text.strip()
        #img_url=img.a.img['src']
        content_url=flex.find('h2','text-xl').a['href'].split('&')
        content_url=url+'&'+content_url[1]+'&'+content_url[2]
        position=flex.find('div','text-base').text.strip()
        altitude=flex.find('ul','space-x-2.5').text.strip()
        print('標題:',title)
        #print('照片網址:',img_url)
        print('內容網址:',content_url)
        print('位置:',position)
        print('標高:',altitude)
        print('照片網址:',img_)
        print()
        page_contents_list.append([title,content_url,position,altitude,img_])

# 先抓到最後一頁 確認總頁數    
driver.find_element(By.CSS_SELECTOR,'.pagination').find_elements(By.CSS_SELECTOR,'a')[11].click()
print(driver.current_url)

# try:
#     driver.find_element(By.CSS_SELECTOR,'#dismiss-button').click()
#     print('已關閉廣告')
# except:
#     print('未出現廣告')

time.sleep(5) # 要等待網頁跑一下才抓到最後頁數
last_page=int(driver.current_url[-3:])
print(last_page)

# 回到第一頁
driver.find_element(By.CSS_SELECTOR,'.pagination').find_elements(By.CSS_SELECTOR,'a')[0].click()
time.sleep(2)

# 每頁抓內容 
# 如果使用pandas一次轉df爬到一半斷掉 可在range(start_page=1)這裡改斷掉的頁面數字 list會紀錄好從這裏開始 並註解一開始為空的page_contents_list=[]
for last in range(start_page,last_page):
    page_url=url+'&page='+str(last)
    print('目前網址:',page_url)
    driver.get(page_url)
    Get_page_content_fn()
    time.sleep(3)
    page_imgs_list.clear()  # 清空放照片的list

columns_name=['標題','內容網址','位置','標高','照片網址']
contents_df=pd.DataFrame(page_contents_list,columns=columns_name)

contents_df.to_csv('hiking_mountains.csv')

driver.close() 


In [13]:
# 邊爬邊存csv 照片存取是錯的 都只存第一頁 page_imgs_list.clear()解決 上面要重寫

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import bs4,os,time,csv

# 啟動瀏覽器工具的選項
options = webdriver.ChromeOptions()
# options.add_argument("--headless")                # 不開啟實體瀏覽器背景執行
options.add_argument("--incognito")               # 開啟無痕模式
options.add_argument("--disable-popup-blocking") # 禁用彈出攔截

columns_names=['標題','內容網址','位置','標高','照片網址']
page_imgs_list,page_contents_list=[],[]
page_contents_list.append(columns_names)
start_page=0 # 如果中途斷掉 將此改為斷掉的頁數 並註解上方page_contents_list


url='https://hiking.biji.co/index.php?q=mountain'
# #'&page='
# driver_path=Service('D:/chromedriver.exe')
# driver=webdriver.Chrome(service=driver_path)
# driver.get(url)

driver_url='https://hiking.biji.co/'
driver_path=Service('D:/chromedriver.exe')
driver=webdriver.Chrome(service=driver_path)
driver.get(driver_url)
time.sleep(4)

hh=driver.find_elements(By.CSS_SELECTOR,'.justify-center')[0].find_element(By.CSS_SELECTOR,'a[title="路線"]')
hh.click()
time.sleep(4)
oo=driver.find_element(By.CSS_SELECTOR,'.justify-center .bg-gray-100 a[href="/index.php?q=mountain"]')
oo.click()
time.sleep(2)


# 去抓個頁面的內容
def Get_page_content_fn():
    driver_soup=bs4.BeautifulSoup(driver.page_source,'lxml')  # 在def外就只會一直抓第一頁
    find_imgs=driver_soup.find('div',id='mountain_list').find_all('div','flex')
    find_flexs=driver_soup.find('div',id='mountain_list').find_all('div','flex-1')  # 要注意div內是否還有一樣的屬性名稱
    print(len(find_flexs))
    print(len(find_imgs))
    
    for i in range(0,len(find_imgs),2): 
        img_url=find_imgs[i].a.img['data-src']  # 選擇完整網址的屬性'data-src' 比'src'不易有bug
        #print(i,img_url)
        page_imgs_list.append(img_url)
    for flex,img_ in zip(find_flexs,page_imgs_list):  # 放入檔案後要清空放照片的page_imgs_list 才不會永遠只抓前20筆
        #print(flex.text.strip())
        title=flex.h2.text.strip()
        #img_url=img.a.img['src']
        content_url=flex.find('h2','text-xl').a['href'].split('&')
        content_url=url+'&'+content_url[1]+'&'+content_url[2]
        position=flex.find('div','text-base').text.strip()
        altitude=flex.find('ul','space-x-2.5').text.strip()
        print('標題:',title)
        #print('照片網址:',img_url)
        print('內容網址:',content_url)
        
        print('位置:',position)
        print('標高:',altitude)
        print('照片網址:',img_)
        print()
        page_contents_list.append([title,content_url,position,altitude,img_])

    # 寫入檔案  寫'a'會變成一職累加進到csv???
    with open('Csvs/hiking_mountains_writerow.csv','w',newline='',encoding='utf-8') as csv_file:
        csv_writer=csv.writer(csv_file)
        for j in page_contents_list:  # 二維list的每次寫入一筆list資料
            csv_writer.writerow(j)

# 先抓到最後一頁 確認總頁數    
driver.find_element(By.CSS_SELECTOR,'.pagination').find_elements(By.CSS_SELECTOR,'a')[11].click()
print(driver.current_url)

# try:
#     driver.find_element(By.CSS_SELECTOR,'#dismiss-button').click()
#     print('已關閉廣告')
# except:
#     print('未出現廣告')

time.sleep(5) # 要等待網頁跑一下才抓到最後頁數
last_page=int(driver.current_url[-3:])
print(last_page)

# 回到第一頁
driver.find_element(By.CSS_SELECTOR,'.pagination').find_elements(By.CSS_SELECTOR,'a')[0].click()
time.sleep(2)

# 每頁抓內容
print(last_page)
for last in range(start_page,last_page+1):
    page_url=url+'&page='+str(last)
    print('目前網址:',page_url)
    driver.get(page_url)
    Get_page_content_fn()
    time.sleep(8)
    page_imgs_list.clear()  # 清空放照片的list

driver.close() 

https://hiking.biji.co/index.php?q=mountain&page=154
154
154
目前網址: https://hiking.biji.co/index.php?q=mountain&page=0
20
40
標題: 槓子寮山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=103
位置: 基隆市信義區,基隆市中正區
標高: 標高 163公尺
照片網址: https://cdntwrunning.biji.co/300_3c9c6596bd28af286ae4beb5aab1f1a5f7c418295da5ef61f1fc648b677a5254.jpg

標題: 紅淡山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=106
位置: 基隆市仁愛區
標高: 標高 208公尺
照片網址: https://cdntwrunning.biji.co/300_b35dc02e4de41d831571018161276523796723d18b87ddd6c5e57a5387a31106.jpg

標題: 三角洲嶺北峰
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=684
位置: 基隆市仁愛區
標高: 標高 115公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 三角洲嶺山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=685
位置: 基隆市仁愛區
標高: 標高 117公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 獅球嶺山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=686
位置: 基隆市仁愛區
標高: 標高 125公尺


目前網址: https://hiking.biji.co/index.php?q=mountain&page=3
20
40
標題: 中埔山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=222
位置: 台北市大安區,台北市文山區
標高: 標高 139公尺
照片網址: https://cdntwrunning.biji.co/300_be6f43fbf3024bc01649de36c74f1df976744420b93213e73a34aa1e021a8746.jpg

標題: 中埔山東峰
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=487
位置: 台北市大安區,
標高: 標高 137公尺
照片網址: https://cdntwrunning.biji.co/300_ef66ab41ece226c55a0929cac376022f.jpg

標題: 南港山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=112
位置: 台北市信義區, 台北市南港區,
標高: 標高 375公尺
照片網址: https://cdntwrunning.biji.co/300_b1a44a58f743f39c7979aa832051a7bb134890b39571464b8e521142b26211ac.jpg

標題: 虎山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=472
位置: 台北市信義區
標高: 標高 144公尺
照片網址: https://cdntwrunning.biji.co/300_669ac2ff9e8c1fc0638ccc46bfdbb8ea9613f8d893ded7fa6134dedbb8562652.jpg

標題: 豹山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=473
位置: 台北市信義區
標高: 標高 141公尺
照片網址: https://cd

目前網址: https://hiking.biji.co/index.php?q=mountain&page=6
20
40
標題: 中正山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=220
位置: 台北市北投區
標高: 標高 646公尺
照片網址: https://cdntwrunning.biji.co/300_5de61c3e70a8fecc7d383d4f869c0dd778d7e37320a662c140b2010d347057de.jpg

標題: 向天山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=226
位置: 台北市北投區
標高: 標高 949公尺
照片網址: https://cdntwrunning.biji.co/300_63d4a0574b47526cfc4c2a6b14f55ca19e774e59ea8af0bb76da4cb627edc309.jpg

標題: 七星山東峰
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=229
位置: 台北市北投區
標高: 標高 1,108公尺
照片網址: https://cdntwrunning.biji.co/300_7842501a2014b1068c2fdee4af7c629da853b41d225acb3d970d48df81cab79c.jpg

標題: 大屯山西峰
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=230
位置: 台北市北投區
標高: 標高 982公尺
照片網址: https://cdntwrunning.biji.co/300_1aa458602a57530d381dad4b45cab99bcee094fe448363365965b2088122f3a4.jpg

標題: 大屯山南峰
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=231
位置: 台北市北投區
標高: 標高

目前網址: https://hiking.biji.co/index.php?q=mountain&page=9
20
40
標題: 梘頭山南峰
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=7181
位置: 台北市內湖區
標高: 標高 35公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 石崁山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=7183
位置: 台北市內湖區
標高: 標高 340公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 聖明山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=7202
位置: 台北市內湖區
標高: 標高 86公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 論劍山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=7263
位置: 台北市內湖區
標高: 標高 266公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 牛稠湖山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=7556
位置: 台北市內湖區
標高: 標高 126公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 南港山南峰
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&

目前網址: https://hiking.biji.co/index.php?q=mountain&page=12
20
40
標題: 阿泉坑山西北峰
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=7550
位置: 台北市文山區
標高: 標高 84公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 貓空尖
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=7627
位置: 台北市文山區
標高: 標高 556公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 貓空圓山東北峰
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=7628
位置: 台北市文山區
標高: 標高 477公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 獅球山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=7629
位置: 台北市文山區
標高: 標高 283公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 軍功山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=7630
位置: 台北市文山區
標高: 標高 92公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 金頭山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=det

...

目前網址: https://hiking.biji.co/index.php?q=mountain&page=142
20
40
標題: 三角錐山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=753
位置: 花蓮縣秀林鄉,
標高: 標高 2,607公尺
照片網址: https://cdntwrunning.biji.co/300_1f66da4d8f94c929c7ada51ce9b9bad3.jpg

標題: 卡拉寶山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=782
位置: 花蓮縣秀林鄉
標高: 標高 2,397公尺
照片網址: https://cdntwrunning.biji.co/300_ddddfc1c23323b36139638e98c4718d0.jpg

標題: 鯉魚山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=187
位置: 花蓮縣壽豐鄉,
標高: 標高 601公尺
照片網址: https://cdntwrunning.biji.co/300_d0ee2c484d78243c655b212a223a20ac.jpeg

標題: 月眉山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=188
位置: 花蓮縣壽豐鄉,
標高: 標高 614公尺
照片網址: https://cdntwrunning.biji.co/300_92a86266b9344c4f3ae20d8020bf4c5d.jpg

標題: 八里灣山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=189
位置: 花蓮縣豐濱鄉,花蓮縣瑞穗鄉
標高: 標高 924公尺
照片網址: https://cdntwrunning.biji.co/300_717099cf9f23b02d29310ffd2bcf0027a880d2accd6ab9dfe9bc4f86768879ff.jpg



目前網址: https://hiking.biji.co/index.php?q=mountain&page=145
20
40
標題: 庶把掘
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=7378
位置: 台東縣延平鄉
標高: 標高 2,137公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 2125峰
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=7568
位置: 台東縣延平鄉
標高: 標高 2,125公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 羅試山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=7571
位置: 台東縣延平鄉
標高: 標高 1,098公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 萬山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=7615
位置: 台東縣延平鄉
標高: 標高 2,535公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 檳榔四格山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=638
位置: 台東縣卑南鄉
標高: 標高 410公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 大浦山
內容網址: https://hiking.biji.co/index.php?q=mountain&ac

目前網址: https://hiking.biji.co/index.php?q=mountain&page=148
20
40
標題: 舞樂山前峰
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=6006
位置: 台東縣海端鄉
標高: 標高 1,578公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 真妻子山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=6007
位置: 台東縣海端鄉
標高: 標高 1,322公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 南舞樂山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=6008
位置: 台東縣海端鄉
標高: 標高 1,900公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 羽美山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=6009
位置: 台東縣海端鄉
標高: 標高 1,346公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 三等三角點 7510
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=6010
位置: 台東縣海端鄉
標高: 標高 1,161公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 背戶庵山
內容網址: https://hiking.biji.co/index.php?q

目前網址: https://hiking.biji.co/index.php?q=mountain&page=151
20
40
標題: 安通越山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=5740
位置: 台東縣長濱鄉
標高: 標高 993公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 赤門山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=5805
位置: 台東縣長濱鄉
標高: 標高 255公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 馬久答山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=6255
位置: 台東縣長濱鄉
標高: 標高 1,289公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 赤門溪山東峰
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=6290
位置: 台東縣長濱鄉
標高: 標高 390公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 石坑山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=6291
位置: 台東縣長濱鄉
標高: 標高 413公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 彭子存山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=

目前網址: https://hiking.biji.co/index.php?q=mountain&page=154
14
28
標題: 森永山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=6319
位置: 台東縣達仁鄉
標高: 標高 226公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 茶留凡山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=7617
位置: 台東縣達仁鄉
標高: 標高 780公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 蛇頭山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=198
位置: 澎湖縣馬公市,
標高: 標高 20公尺
照片網址: https://cdntwrunning.biji.co/300_93801a2806cf6a1dd8bcc975b5cffc2fba5a166375cb2a3a9d1cbe81696bfd20.jpg

標題: 豬母水山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=6417
位置: 澎湖縣馬公市
標高: 標高 26公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 桶盤山
內容網址: https://hiking.biji.co/index.php?q=mountain&act=detail&id=6956
位置: 澎湖縣馬公市
標高: 標高 25公尺
照片網址: http://hiking.biji.co/resource/images/default_img/default3v2.jpg

標題: 貓嶼
內容網址: https://hiking.biji.c

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd

# 要換成全部讀取csv的
hh=['https://hiking.biji.co/index.php?q=mountain&act=detail&id=103','https://hiking.biji.co/index.php?q=mountain&act=detail&id=106','https://hiking.biji.co/index.php?q=mountain&act=detail&id=684']
inside_all_list=[]
inside_title_img_list=[]
inside_content_list=[list() for c in range(len(hh))]

driver_path=Service('D:/chromedriver.exe')
driver=webdriver.Chrome(service=driver_path)

# hiking_df=pd.read_csv('Csvs/hiking_mountains.csv')
# #print(hiking_df)

# # 將內容網址那整列轉成list 方便一個一個輸入網頁 抓更多內容
# hiking_content_url=list(hiking_df['內容網址'])


for i,content_url in enumerate(hh):
    driver.get(content_url)
    time.sleep(3)
    
#     inside_title=driver.find_elements(By.CSS_SELECTOR,'#category_base .leading-relaxed dt')
#     for in_title in inside_title:
    inside_title=driver.find_element(By.TAG_NAME,'h1')
    inside_img_url=driver.find_element(By.CSS_SELECTOR,'.main-left img')
    inside_key=driver.find_elements(By.CSS_SELECTOR,'#category_base .leading-relaxed dt')
    inside_content=driver.find_elements(By.CSS_SELECTOR,'#category_base .leading-relaxed dd')
    in_title=inside_title.text.strip()
    in_img_url=inside_img_url.get_attribute('src')
#     print(in_title)
#     print(in_img_url)
    inside_content_list[i].append(in_title)
    inside_content_list[i].append(in_img_url)
    for in_key,in_content in zip(inside_key,inside_content):
        
        print(in_content.text.strip())     
        inside_content_list[i].append(in_content.text.strip())  # inside_content_list[i] 注意i是外面for的i 這樣寫才會成為二維list 每個list個是每筆資料
    print('數量:',len(inside_content_list))
    
    
#     inside_all_list.append(inside_content_list)
    
    print(inside_content_list)
#     print(inside_all_list)

driver.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=101.0.4951.54)
Stacktrace:
Backtrace:
	Ordinal0 [0x0087B8F3+2406643]
	Ordinal0 [0x0080AF31+1945393]
	Ordinal0 [0x006FC748+837448]
	Ordinal0 [0x006E42B0+737968]
	Ordinal0 [0x00746C09+1141769]
	Ordinal0 [0x00753E22+1195554]
	Ordinal0 [0x00744096+1130646]
	Ordinal0 [0x0071E636+976438]
	Ordinal0 [0x0071F546+980294]
	GetHandleVerifier [0x00AE9612+2498066]
	GetHandleVerifier [0x00ADC920+2445600]
	GetHandleVerifier [0x00914F2A+579370]
	GetHandleVerifier [0x00913D36+574774]
	Ordinal0 [0x00811C0B+1973259]
	Ordinal0 [0x00816688+1992328]
	Ordinal0 [0x00816775+1992565]
	Ordinal0 [0x0081F8D1+2029777]
	BaseThreadInitThunk [0x76B7FA29+25]
	RtlGetAppContainerNamedObjectPath [0x773D7A7E+286]
	RtlGetAppContainerNamedObjectPath [0x773D7A4E+238]


In [ ]:
inside_all_list


In [11]:
# 練dict轉df
ff,oo=[],[]

for j in range(0,2):
    gg=[11,22]
    for i in range(0,9):
        ff.append(i)
    gg.extend(ff)
    print(gg)
    
# 在一行行寫入csv 
with open('j.csv','w',newline='',encoding='utf-8') as f:
    csv_w=csv.writer(f)
    for g in range(0,len(gg),3):
        j=gg[g:g+3]
        print(j)
        csv_w.writerow(j)

[11, 22, 0, 1, 2, 3, 4, 5, 6, 7, 8]
[11, 22, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8]
[11, 22, 0]
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]
[1, 2, 3]
[4, 5, 6]
[7, 8]
